In [1]:
__author__ = 'ShuoData: https://www.kaggle.com/shuodata'

#just ignore the warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import sys
import visuals as vs
import renders as rs
from library import *
from cleandf import *
from someclf import *




/home/joaquin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/joaquin/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
df_test = Load_DataFrames('/Data/total.xlsx','DF_2')
df_train = Load_DataFrames('/Data/total.xlsx','DF_1')

In [3]:
from IPython.display import display # Allows the use of display() for DataFrames

display(df_train.head())
display(df_test.head())

,people_id,activity_id,date_t,char_1_t,char_2_t,char_3_t,char_4_t,char_5_t,char_6_t,char_7_t,...,char_33,char_34,char_35,char_36,char_37,char_38,min_date,max_date,apg,ppg
0,ppl_100,act2_1734928,1193,0,0,0,0,0,0,0,...,0,1,1,1,0,36,0,1198,685757,53342
1,ppl_100,act2_2434093,860,0,0,0,0,0,0,0,...,0,1,1,1,0,36,0,1198,685757,53342
2,ppl_100,act2_3404049,860,0,0,0,0,0,0,0,...,0,1,1,1,0,36,0,1198,685757,53342
3,ppl_100,act2_3651215,1171,0,0,0,0,0,0,0,...,0,1,1,1,0,36,0,1198,685757,53342
4,ppl_100,act2_4109017,1193,0,0,0,0,0,0,0,...,0,1,1,1,0,36,0,1198,685757,53342


,people_id,activity_id,date_t,char_1_t,char_2_t,char_3_t,char_4_t,char_5_t,char_6_t,char_7_t,...,char_33,char_34,char_35,char_36,char_37,char_38,min_date,max_date,apg,ppg
0,ppl_100004,act1_249281,791,5,10,5,1,6,1,1,...,1,1,1,1,1,76,791,791,2,1
1,ppl_100004,act2_230855,791,0,0,0,0,0,0,0,...,1,1,1,1,1,76,791,791,2,1
2,ppl_10001,act1_240724,877,12,1,5,4,6,1,1,...,1,1,1,1,1,90,532,931,54,4
3,ppl_10001,act1_83552,921,20,10,5,4,6,1,1,...,1,1,1,1,1,90,532,931,54,4
4,ppl_10001,act2_1043301,878,0,0,0,0,0,0,0,...,1,1,1,1,1,90,532,931,54,4


In [4]:
cols = ['people_id', 'activity_id', 'min_date', 'max_date', 'apg', 'ppg', 'date_t', 'wd_t', 'acts', 'group_1',\
        'wd', 'date','char_1_t', 'char_2_t', 'char_3_t', 'char_4_t', 'char_5_t', 'char_6_t', 'char_7_t',\
        'char_8_t', 'char_9_t', 'char_10_t', 'act', 'ac_1', 'ac_2', 'ac_3', 'ac_4', 'ac_5', 'ac_6', 'ac_7',  \
        'char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 'char_10', \
        'char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16', 'char_17', 'char_18', 'char_19', \
        'char_20', 'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27', 'char_28', \
        'char_29', 'char_30', 'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36', 'char_37', 'char_38']

In [5]:
df_train = df_train[cols + ['outcome']]
df_test = df_test[cols]

In [6]:
#df_train = df_train[df_train['group_1'] != 17304 ]

In [7]:
print "Extracting features and outcome from DataFrame..."
features = df_train.drop(['outcome','people_id','activity_id'], axis=1)
outcome = df_train['outcome']

df_train_droped = df_train.drop(['people_id','activity_id'], axis=1)

df_test_droped = df_test.drop(['people_id','activity_id'], axis=1)

## Produce a matrix for test data
test_data = df_test.values.tolist()
test_droped = df_test_droped.values.tolist()

Extracting features and outcome from DataFrame...


In [8]:
print(df_train_droped.shape)
print(df_test_droped.shape)

(1048575, 67)
(498687, 66)


In [9]:
X_train, y_train = features, outcome

In [10]:
params = {
   # 'loss' : ('deviance', 'exponential'),
    'n_estimators' : [10],
    'max_depth':(60,63),
    'max_features': ['auto']
         }

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

def fit_model2(X, y, __params):
    """ Performs grid search over the 'max_depth' parameter for a
        decision tree regressor trained on the input data [X, y]. """

    __clf = GradientBoostingClassifier()

    # Transform 'performance_metric' into a scoring function using 'make_scorer'
    scoring_fnc = make_scorer(performance_metric, greater_is_better=True)

    # Create the grid search object
    grid = GridSearchCV(__clf, __params, scoring=scoring_fnc, verbose=True, n_jobs=4)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)
    print ("Regresor: {},\t Score: {}".format('RandomForestClassifier',grid.best_score_))

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [12]:
reg2 = fit_model2(X_train, y_train, params)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed: 133.8min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed: 133.8min finished


Regresor: RandomForestClassifier,	 Score: 0.617264992859


In [13]:
from sklearn.externals import joblib
joblib.dump(reg2, 'GBT.pkl') 

['GBT.pkl']

In [14]:
y_pred = reg2.predict(df_test_droped)

In [15]:
CreateSubmission(df_test, y_pred,"GBT")

Saving submission for GBT at 20161112132913...


In [19]:
def CreateSubmission_wo(__df_test, __y_pred,__csf):
    now = datetime.datetime.now()
    activity_id = __df_test['activity_id'].values.tolist()
    group_1 = __df_test['group_1'].values.tolist()
    d = {'activity_id' : activity_id,
     'group_1': group_1,
     'outcome' : __y_pred}
    submission = pd.DataFrame(d)
    #submission.loc[submission['group_1'] == 17304,'outcome' ] = 0

    submission = submission.drop('group_1', axis=1)
    submission['outcome'] = submission['outcome'].astype(int)
    print "Saving submission for {} at {}...".format(__csf, now.strftime("%Y%m%d%H%M%S" ))
    submission.to_csv("./Data/submission-{}-{}.csv".format(now.strftime("%Y%m%d%H%M%S"), __csf), sep=',', index=False)


In [20]:
CreateSubmission_wo(df_test, y_pred,"GBT")

Saving submission for GBT at 20161112174802...
